In [2]:
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [14]:
# !pip install pyLDAvis


### Preparing the Data

In [3]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [4]:
stopwords = stopwords.words("english")

In [5]:
print (stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [8]:
data = load_data("ushmm_dn.json")["texts"]

print (data[0][0:100])

 My name David Kochalski. I was born in a small town called , and I was born May 5, 1928.  Well, we 


In [20]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:100])



name bear small town call bear very hard work child father mother small mill flour buckwhe


In [21]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])



['name', 'bear', 'small', 'town', 'call', 'bear', 'very', 'hard', 'work', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat', 'prosperous', 'comfortable', 'go', 'school']


In [22]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 2), (1, 11), (2, 1), (3, 2), (4, 1), (5, 2), (6, 1), (7, 2), (8, 3), (9, 1), (10, 12), (11, 1), (12, 8), (13, 1), (14, 2), (15, 1), (16, 3), (17, 2), (18, 1), (19, 2)]
able


In [23]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")



### Vizualizing the Data

In [25]:


pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis



C:\Users\soulo\anaconda3\lib\site-packages\sklearn\manifold\_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.133190  0.384314       1        1  20.580546
2      0.081535  0.260138       2        1  18.077433
3      0.026515  0.348031       3        1  13.693676
6     -0.017176  0.337688       4        1   9.876425
24     0.229294  0.339373       5        1   9.511923
18     0.161103  0.360121       6        1   8.989074
17     0.078985  0.353031       7        1   7.929111
20    -0.148494  0.315812       8        1   6.216059
28    -0.063025  0.416683       9        1   3.467672
16     0.275564  0.276356      10        1   1.652289
22    -0.037868 -0.169550      11        1   0.000318
14    -0.037870 -0.169556      12        1   0.000314
13    -0.037872 -0.169565      13        1   0.000307
5     -0.037873 -0.169572      14        1   0.000300
27    -0.037874 -0.169575      15        1   0.000299
26    -0.037874 -0.169575      16        1   0.000297
1     -0.037874 -0.169576      17        1   0.000296
8     -0.037875 -0.169578      18        1   0.000295
10    -0.037875 -0.169578      19        1   0.000295
19    -0.037875 -0.169579      20        1   0.000293
29    -0.037875 -0.169581      21        1   0.000287
9     -0.037875 -0.169581      22        1   0.000286
23    -0.037875 -0.169581      23        1   0.000285
21    -0.037876 -0.169583      24        1   0.000277
25    -0.037876 -0.169584      25        1   0.000275
7     -0.037877 -0.169587      26        1   0.000274
11    -0.037876 -0.169586      27        1   0.000274
15    -0.037876 -0.169586      28        1   0.000273
4     -0.037877 -0.169587      29        1   0.000273
12    -0.037877 -0.169588      30        1   0.000273, topic_info=       Term          Freq         Total Category  logprob  loglift
443    know  21190.000000  21190.000000  Default  30.0000  30.0000
337     get  10882.000000  10882.000000  Default  29.0000  29.0000
804   there  11607.000000  11607.000000  Default  28.0000  28.0000
736      so  21207.000000  21207.000000  Default  27.0000  27.0000
344      go  24475.000000  24475.000000  Default  26.0000  26.0000
..      ...           ...           ...      ...      ...      ...
564   other      0.000166   5135.644781  Topic30  -9.6392  -4.4360
66     back      0.000166   5122.551537  Topic30  -9.6392  -4.4335
341    give      0.000166   4803.830719  Topic30  -9.6393  -4.3693
466  little      0.000166   4664.343850  Topic30  -9.6393  -4.3399
862     war      0.000166   3060.036171  Topic30  -9.6393  -3.9183

[3098 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.061289     able
0         2  0.426318     able
0         3  0.040559     able
0         4  0.059486     able
0         5  0.001803     able
...     ...       ...      ...
894      10  0.032165     year
8660      5  0.080349  yeshiva
8660      7  0.964187  yeshiva
5878      5  0.068672       za
5878      7  0.892741       za

[2782 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 4, 7, 25, 19, 18, 21, 29, 17, 23, 15, 14, 6, 28, 27, 2, 9, 11, 20, 30, 10, 24, 22, 26, 8, 12, 16, 5, 13])